# Comparative Analysis of TMP Databases

**Project:** Digital Teotihuacan Mapping Project (TMP) - Phase 1

**Objective:** This notebook synthesizes the results from the entire profiling pipeline to conduct a comparative analysis of the four legacy databases and the two wide-format benchmark databases. Its primary goal is to use quantitative data to compare these database architectures on three key axes: **Structural Complexity**, **Resource Usage**, and **Query Performance**. 

The findings from this notebook will directly inform the final recommendation for the Phase 2 unified database architecture.

--- 
## 1. Setup and Configuration

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from IPython.display import display, Markdown
import plotly.express as px
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler

# --- Path Definitions ---
# Assumes the notebook is run from the 'notebooks' directory.
PROJECT_ROOT = Path.cwd().parent.parent
REPORTS_DIR = PROJECT_ROOT / "outputs" / "reports"

# --- Styling and Display Options ---
pd.set_option('display.max_columns', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

def display_header(title):
    display(Markdown(f'### {title}'))

print("✅ Setup complete.")
print(f"Reports Directory: {REPORTS_DIR}")

--- 
## 2. Data Loading
We load two key outputs from the `04_run_comparison.py` script:
1. `comparison_matrix.csv`: High-level summary metrics.
2. `report_performance_summary_detailed.csv`: The enriched, long-form performance data with calculated comparative metrics.

In [ ]:
matrix_path = REPORTS_DIR / 'comparison_matrix.csv'
perf_path = REPORTS_DIR / 'report_performance_summary_detailed.csv'

if not matrix_path.exists() or not perf_path.exists():
    raise FileNotFoundError(f"Critical Error: Report files not found in {REPORTS_DIR}. Please run the 04_run_comparison.py script first.")

# Load the matrix and transpose it so databases are rows
comparison_df = pd.read_csv(matrix_path, index_col=0).T.reset_index().rename(columns={'index': 'Database'})

# Load the detailed performance data
perf_summary_df = pd.read_csv(perf_path)

print("Loaded Comparison Matrix:")
display(comparison_df)

print("\nLoaded Detailed Performance Summary Data:")
display(perf_summary_df.head())

--- 
## 3. High-Level Comparison Matrix
A styled view of the main comparison matrix. Color gradients highlight high/low values for each metric, providing an at-a-glance summary.
- <span style='color: #440154;'>**Purple/Dark**</span>: Higher values
- <span style='color: #fde725;'>**Yellow/Light**</span>: Lower values

In [ ]:
display_header("Styled Comparison Matrix")

styled_df = comparison_df.style.background_gradient(cmap='viridis', axis=0)\
    .set_caption("Comparative Database Metrics")\
    .format('{:.2f}', subset=pd.IndexSlice[:, ['Database Size (MB)', 'JDI (Join Dependency Index)', 'NF (Normalization Factor)']])\
    .format('{:,.0f}', subset=pd.IndexSlice[:, ['Table Count', 'View Count', 'Total Estimated Rows', 'Total Index Count', 'LIF (Logical Interop. Factor)']])

display(styled_df)

--- 
## 4. Structural Complexity Analysis
This section focuses on the metrics that quantify the relational complexity and degree of normalization of the legacy schemas.

In [ ]:
display_header("Schema Complexity Metrics (Legacy Databases)")

complexity_metrics = [
    'Database', 'Table Count', 
    'JDI (Join Dependency Index)', 
    'LIF (Logical Interop. Factor)',
    'NF (Normalization Factor)'
]
# Filter for legacy DBs only, as these metrics don't apply to the single-table benchmarks
legacy_df = comparison_df[~comparison_df['Database'].str.contains('benchmark')]
display(legacy_df[complexity_metrics])

# --- Advanced Visualization: Complexity Radar Plot ---
radar_metrics = ['Table Count', 'JDI (Join Dependency Index)', 'NF (Normalization Factor)']
radar_df = legacy_df[['Database'] + radar_metrics].copy()

# Normalize metrics to a 0-1 scale for fair comparison on the radar plot
scaler = MinMaxScaler()
radar_df[radar_metrics] = scaler.fit_transform(radar_df[radar_metrics])

fig = go.Figure()

for index, row in radar_df.iterrows():
    fig.add_trace(go.Scatterpolar(
        r=row[radar_metrics].values,
        theta=radar_metrics,
        fill='toself',
        name=row['Database']
    ))

fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True,
      range=[0, 1]
    )),
  showlegend=True,
  title='Normalized Complexity Profile of Legacy Databases'
)

fig.show()

--- 
## 5. Query Performance Deep Dive
This is the most critical comparison. It directly measures the analytical query performance of the legacy normalized schemas against the denormalized wide-format benchmark schemas using the pre-calculated metrics from the `04_run_comparison.py` script.

In [ ]:
display_header("Schema Efficiency Factor by Query Category (Log Scale)")

if not perf_summary_df.empty:
    fig = px.bar(perf_summary_df,
                 x='database', 
                 y='schema_efficiency_factor', 
                 color='category', 
                 barmode='group',
                 title='Schema Efficiency Factor (Lower is Better)',
                 labels={'schema_efficiency_factor': 'Efficiency Factor (Log Scale)', 'database': 'Database'},
                 category_orders={'category': ['baseline', 'join_performance', 'complex_filtering']})
    
    fig.update_yaxes(type="log") # Use a log scale as differences can be huge
    fig.add_hline(y=1.0, line_dash="dot", annotation_text="Benchmark Baseline", annotation_position="bottom right")
    fig.update_layout(height=600)
    fig.show()
else:
    print("No performance data to plot.")

In [ ]:
display_header("Performance Improvement vs. Best Benchmark")

if not perf_summary_df.empty:
    # Filter out the baseline databases themselves for a cleaner plot
    improvement_df = perf_summary_df[~perf_summary_df['database'].str.contains('benchmark')].copy()
    
    fig = px.bar(improvement_df.sort_values('performance_improvement_factor'),
                 x='query_id', 
                 y='performance_improvement_factor', 
                 color='database', 
                 facet_row='category',
                 barmode='group',
                 title='Performance Improvement of Benchmark Schemas vs. Legacy Schemas',
                 labels={'performance_improvement_factor': '% Improvement vs. Benchmark', 'query_id': 'Query ID'})
    
    fig.update_layout(height=800)
    fig.show()
else:
    print("Could not generate performance improvement plot.")

--- 
## 6. Qualitative Architectural Trade-offs
The quantitative data above supports a qualitative assessment of the architectural trade-offs between the legacy design and the proposed wide-format design.

| Feature                  | Legacy Normalized (e.g., DF9)      | Proposed Wide-Format (Benchmark)   | Justification Based on Data                                                                      |
| :----------------------- | :--------------------------------- | :--------------------------------- | :----------------------------------------------------------------------------------------------- |
| **Query Performance** | `Low`                              | `High`                             | The 'Schema Efficiency Factor' chart shows legacy databases are multiple times slower.             |
| **Storage Cost** | `Low`                              | `High`                             | `comparison_matrix.csv` shows benchmark DBs are larger due to data duplication.                |
| **Schema Complexity** | `High` (High JDI/NF, Many Tables)  | `Very Low` (1 Table)               | The complexity radar plot visually confirms the high complexity scores of the legacy schemas.    |
| **Data Redundancy** | `Low` (Normalized)                 | `High` (Denormalized)              | This is the inherent trade-off of the wide-format design; we trade storage for speed.            |
| **Ease of Use for BI/GIS** | `Low` (Requires complex joins)     | `High` (Single table source)       | A single flat table is trivial to connect to tools like QGIS, Tableau, or Power BI.              |

---
## 7. Final Analyst Summary & Recommendation

**Instructions:** Based on the comparative analysis, synthesize the findings and provide a formal recommendation for the Phase 2 unified database architecture. This summary will be a primary input for the final white paper.

### Overarching Conclusion:
* *Start with a concise, definitive statement. Example: "The comparative analysis demonstrates conclusively that the highly normalized structure of the legacy databases, particularly `tmp_df9`, is quantitatively inferior for the project's analytical objectives compared to a denormalized, wide-format architecture."*

### Justification from Evidence:
1.  **On Performance:**
    * *Quantify the performance difference. Reference the 'Schema Efficiency Factor' chart directly. Example: "As shown in the efficiency factor plot, the legacy schemas are between 5x and 50x slower for join-heavy queries than the wide-format benchmarks. This performance gap makes interactive analysis on the normalized schemas untenable."*
2.  **On Complexity:**
    * *Reference the complexity metrics and the radar plot. Example: "The legacy schemas exhibit high JDI and NF scores, indicative of significant relational complexity that increases the cognitive load for analysts and the technical barrier for connecting to BI and GIS tools. The radar plot clearly visualizes `tmp_df9` as the most complex outlier."*
3.  **On The Cost/Benefit Trade-off:**
    * *Acknowledge the trade-offs identified in the qualitative table. Example: "While the wide-format approach increases storage costs due to data redundancy, this trade-off is strategically acceptable. The cost of storage is minimal compared to the significant gains in query performance and the drastic reduction in development time and analytical friction for end-users."*

### Formal Recommendation:
* *State the final recommendation clearly and unambiguously.*
* **Recommended Architecture:** "It is the formal recommendation of this analysis that Phase 2 of the Digital TMP project proceeds with the development of a single, denormalized, wide-format primary analytical table. This table should be based on the schema of the `tmp_benchmark_wide_text_nulls` database, as it provides the best balance of performance and human-readability."*
* **Next Steps:** "The next step should be to finalize the schema of this wide-format table, including data type assignments and column naming conventions, and to proceed with the development of the full ETL pipeline in Phase 2 to migrate all legacy data into this new structure."*